In [ ]:
import ast
import json
import numpy as np
import pandas as pd

In [ ]:
parquet = pd.read_parquet('demo_data/so100_strawberry_grape/data/chunk-000/episode_000000.parquet')

In [ ]:
parquet

In [ ]:
parquet['action'].values[0]

In [ ]:
parquet['observation.state'].values[0]

In [ ]:
parquet1 = pd.read_parquet('demo_data/so100_strawberry_grape/data/chunk-000/episode_000001.parquet')

In [ ]:
parquet1

In [ ]:
parquet39 = pd.read_parquet('demo_data/so100_strawberry_grape/data/chunk-000/episode_000039.parquet')

In [ ]:
parquet39

In [ ]:
# !mkdir -p demo_data/elephant_robot/
# !mkdir -p demo_data/elephant_robot/data/chunk-000
# !mkdir -p demo_data/elephant_robot/meta
# !mkdir -p demo_data/elephant_robot/videos/chunk-000/observation.images.webcam
# !cp demo_data/so100_strawberry_grape/meta/* demo_data/elephant_robot/meta/
# # Change the json files in `demo_data/elephant_robot/meta/`


In [ ]:
!pip install openpyxl

In [ ]:
data = pd.read_excel('demo_data/elephant_robot/夹取端子训练.xlsx')

In [ ]:
data

In [ ]:
# 定义要合并的列名
array_columns = ["左臂关节数据", "右臂关节数据"]
numeric_columns = ["左臂力控夹爪位置", "右臂力控夹爪位置"]

# 解析字符串形式的数组
def parse_array_string(array_str):
    if pd.isna(array_str):
        return []
    
    try:
        # 尝试使用ast.literal_eval解析(更安全)
        return ast.literal_eval(array_str)
    except (ValueError, SyntaxError):
        try:
            # 尝试使用json.loads解析
            return json.loads(array_str)
        except json.JSONDecodeError:
            # 如果两种方法都失败，尝试简单的字符串处理
            cleaned_str = array_str.strip('[]').split(',')
            return [float(x.strip()) for x in cleaned_str if x.strip()]
    except:
        # 所有方法都失败时返回空列表
        return []

def merge_data(row):
    result = []
    
    # 添加数组类型数据
    for col in array_columns:
        if isinstance(row[col], list):
            result.extend(row[col])
        elif isinstance(row[col], np.ndarray):
            result.extend(row[col].tolist())
        elif isinstance(row[col], str):
            array_data = parse_array_string(row[col])
            result.extend(array_data)
        # 处理可能的NaN值
        elif pd.isna(row[col]):
            pass  # 或者添加一些默认值
    
    # 添加数字类型数据
    for col in numeric_columns:
        if not pd.isna(row[col]):  # 确保不是NaN
            result.append(row[col])
    
    if len(result) != 16:
        print(len(result))
        print(row)
        
    return result

In [ ]:
data = data.ffill()  # 填充部分空数据
data['observation.state'] = data.apply(merge_data, axis=1)

In [ ]:
data

In [ ]:
data['task_index'] = 0

In [ ]:
with open('demo_data/elephant_robot/夹取端子训练.txt', 'r') as f:
    lines = f.readlines()

# 解析时间戳
timestamps = []
for line in lines:
    try:
        end_time = float(line.strip())
        timestamps.append(end_time)
    except ValueError:
        print(f"警告：无法解析时间戳 '{line.strip()}'，已跳过")

# 确保时间戳是按顺序排列的
timestamps.sort()

print(timestamps)

# 将秒转换为毫秒
timestamps_ms = [t * 1000 for t in timestamps]
print(timestamps_ms)

In [ ]:
# 创建一个函数来确定每行数据属于哪个episode
def assign_episode(time_ms):
    # 为时间戳添加起始值0
    all_timestamps = [0] + timestamps_ms
    
    for i in range(len(all_timestamps)-1):
        if all_timestamps[i] <= time_ms < all_timestamps[i+1]:
            return i
    
    # 如果时间超过最后一个时间戳，归入最后一个episode
    return len(all_timestamps) - 1

# 应用函数创建episode列
data['episode_index'] = data['时间间隔'].cumsum().apply(assign_episode)

data['index'] = list(range(data.shape[0]))

# 将DataFrame拆分成多个episode
episodes = {}
for episode_num in range(len(timestamps) + 1):  # +1 是因为包括timestamps之后的数据
    episodes[episode_num] = data[data['episode_index'] == episode_num].copy()

In [ ]:
len(episodes)

In [ ]:
for episode_num in range(len(timestamps) + 1):
    print(episodes[episode_num].shape[0])

In [ ]:
def compute_actions(episode_df):
    # 假设"合并数据"列包含了完整state
    states = np.array(episode_df["observation.state"].tolist())
    
    # 计算相邻时间步的状态差作为动作
    # 使用t+1时刻减去t时刻的状态
    actions = states[1:] - states[:-1]
    
    # 第一个时间步没有前一状态，可以填充为零或NaN
    first_action = np.zeros_like(actions[0]) if len(actions) > 0 else np.array([])
    all_actions = np.vstack([first_action, actions]) if len(actions) > 0 else np.array([first_action])
    
    return all_actions

In [ ]:

# 为每个episode计算动作
for episode_num, episode_df in episodes.items():
    actions = compute_actions(episode_df)
    episodes[episode_num]["action"] = list(actions)
    
    time_intervals = episode_df["时间间隔"].values
    time_intervals[0] = 0
    cumulative_time_ms = np.cumsum(time_intervals)  # 计算累积和（毫秒）
    episodes[episode_num]["timestamp"] = cumulative_time_ms / 1000.0
    
    # episodes[episode_num]["frame_index"] = list(range(episode_df.shape[0]))
    
    timestamps = episodes[episode_num]["timestamp"].values
        
    # 检测时间间隙(超过1.5倍正常帧间隔)
    fps = 30
    frame_interval = 1.0/fps
    time_diffs = np.diff(timestamps)
    gaps = np.where(time_diffs > 1.5 * frame_interval)[0]
    
    if len(gaps) > 0:
        print(f"Episode {episode_num} 包含 {len(gaps)} 个时间间隙")
        
    # 基本帧索引计算
    frame_indices = np.floor(timestamps * fps).astype(int)
    
    # 可选：确保帧索引连续(无重复)
    unique_indices = []
    last_index = -1
    
    for idx in frame_indices:
        if idx > last_index:
            unique_indices.append(idx)
            last_index = idx
        else:
            # 如果遇到重复或倒退的索引，递增处理
            unique_indices.append(last_index + 1)
            last_index += 1
            
    episodes[episode_num]["frame_index"] = unique_indices

In [ ]:
episodes[0]

In [ ]:
episodes[1]

In [ ]:
for episode_num, episode_df in episodes.items():
    episode_df[['action', 'observation.state', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index']].reset_index().to_parquet(f'demo_data/elephant_robot/data/chunk-000/episode_{episode_num:06d}.parquet')

In [ ]:
from gr00t.utils.misc import any_describe
from gr00t.data.dataset import LeRobotSingleDataset
from gr00t.experiment.data_config import DATA_CONFIG_MAP

dataset_path = "./demo_data/elephant_robot"   # change this to your dataset path

data_config = DATA_CONFIG_MAP["elephant_robot"]

dataset = LeRobotSingleDataset(
    dataset_path=dataset_path,
    modality_configs=data_config.modality_config(),
    embodiment_tag="new_embodiment",
    video_backend="torchvision_av",
)

In [ ]:
resp = dataset[7]
any_describe(resp)

In [ ]:
# visualize the dataset
# show img
import matplotlib.pyplot as plt

images_list = []

for i in range(10):
    resp = dataset[i]
    img = resp["video.webcam"][0]
    images_list.append(img)

fig, axs = plt.subplots(2, 5, figsize=(20, 10))
for i, ax in enumerate(axs.flat):
    ax.imshow(images_list[i])
    ax.axis("off")
    ax.set_title(f"Image {i}")
plt.tight_layout() # adjust the subplots to fit into the figure area.
plt.show()

```
python scripts/gr00t_finetune.py \
   --dataset-path getting_started/demo_data/elephant_robot/ \
   --num-gpus 1 \
   --batch_size 16 \
   --dataloader_num_workers 8 \
   --output-dir ~/elephant_robot-checkpoints  \
   --max-steps 2000 \
   --data-config elephant_robot \
   --video-backend torchvision_av
```